# Guardian Route: Interactive Safety-Weighted Routing

Generate safer routes through Denver using predictive crime risk modeling.

## Overview

This notebook:
1. Loads the trained Cynet model
2. Predicts 4-hour crime risk across Denver tiles
3. Applies risk weights to street network edges
4. Generates safe routes vs. fastest routes
5. Visualizes results on an interactive map

## Setup

In [ ]:
import sys
sys.path.append('../scripts')

import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import folium
import pickle
import joblib
import json
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import custom utilities
from utils.routing import (
    apply_risk_weights,
    find_safe_route,
    calculate_route_metrics,
    compare_routes,
    calculate_risk_reduction,
    route_to_geodataframe,
    export_route_geojson,
    geocode_address
)
from utils.cynet_wrapper import predict_next_4_hours

print("="*60)
print("GUARDIAN ROUTE - Interactive Routing")
print("="*60)
print("✓ Libraries loaded")

## 1. Load Model and Network

In [ ]:
# Load trained model
print("Loading trained model...")
model = joblib.load('../models/trained/cynet_model.pkl')

# Load metadata
with open('../models/trained/model_metadata.json', 'r') as f:
    metadata = json.load(f)

print(f"Model type: {type(model).__name__}")
print(f"Training date: {metadata['training_date']}")
print(f"Data range: {metadata['data_range']}")

if 'metrics' in metadata and metadata['metrics']:
    print(f"\nModel metrics:")
    for key, value in metadata['metrics'].items():
        if value is not None and key != 'note':
            print(f"  {key}: {value:.4f}")

print("✓ Model loaded")

In [ ]:
# Load street network
print("Loading Denver street network...")
G = ox.load_graphml('../data/network/denver_network.graphml')
print(f"Network: {len(G.nodes):,} nodes, {len(G.edges):,} edges")
print("✓ Network loaded")

In [ ]:
# Load tile-edge mapping
print("Loading tile-edge mapping...")
with open('../data/network/tile_edge_mapping.pkl', 'rb') as f:
    tile_edge_mapping = pickle.load(f)
print(f"Mapping: {len(tile_edge_mapping):,} edges mapped to tiles")
print("✓ Mapping loaded")

In [ ]:
# Load spatial grid
print("Loading spatial grid...")
tiles_gdf = gpd.read_file('../data/processed/spatial_grid.geojson')
print(f"Grid: {len(tiles_gdf):,} tiles")
print("✓ Grid loaded")

## 2. USER INPUT: Origin and Destination

**Edit the cells below to set your route:**

In [ ]:
# ========== EDIT THESE VALUES ==========

# Option 1: Use addresses (recommended)
origin_address = "Denver Union Station, Denver, CO"
dest_address = "Denver Art Museum, Denver, CO"

# Option 2: Use lat/lon coordinates (uncomment to use)
# origin_coords = (39.7539, -104.9979)  # (lat, lon)
# dest_coords = (39.7370, -104.9892)

# Departure time (for risk prediction window)
departure_time = datetime.now()  # Or specify: datetime(2025, 1, 15, 14, 0)

# ========================================

In [ ]:
# Geocode addresses to coordinates
print("Geocoding addresses...")

if 'origin_address' in locals():
    print(f"Origin: {origin_address}")
    origin_point = geocode_address(origin_address)
    print(f"  → {origin_point}")
else:
    origin_point = origin_coords
    print(f"Origin: {origin_point}")

if 'dest_address' in locals():
    print(f"Destination: {dest_address}")
    dest_point = geocode_address(dest_address)
    print(f"  → {dest_point}")
else:
    dest_point = dest_coords
    print(f"Destination: {dest_point}")

print(f"\nDeparture time: {departure_time.strftime('%Y-%m-%d %H:%M')}")
print("✓ Locations ready")

## 3. Predict 4-Hour Crime Risk

In [ ]:
# Generate risk predictions
print(f"Predicting crime risk for next 4 hours from {departure_time.strftime('%H:%M')}...")

tile_risks = predict_next_4_hours(model, tiles_gdf, reference_time=departure_time)

print(f"\nPredictions generated for {len(tile_risks):,} tiles")
print(f"Average risk: {sum(tile_risks.values())/len(tile_risks):.4f}")
print(f"Max risk: {max(tile_risks.values()):.4f}")
print("✓ Risk predictions complete")

In [ ]:
# Display top 10 riskiest tiles
risk_df = pd.DataFrame(list(tile_risks.items()), columns=['tile_id', 'risk_prob'])
risk_df = risk_df.sort_values('risk_prob', ascending=False)

print("\nTop 10 Riskiest Tiles (Next 4 Hours):")
print("="*50)
for idx, row in risk_df.head(10).iterrows():
    print(f"{row['tile_id']}: {row['risk_prob']:.4f}")

## 4. Compute Routes

In [ ]:
# Apply risk weights to network
print("Applying risk weights to street network...")
G_risk = apply_risk_weights(G.copy(), tile_edge_mapping, tile_risks)
print("✓ Risk weights applied")

In [ ]:
# Find routes
print("\nCalculating routes...")

# Safe route (minimize risk)
print("  - Safe route (minimize risk_weight)...")
safe_route = find_safe_route(G_risk, origin_point, dest_point, weight='risk_weight')

# Fastest route (minimize distance)
print("  - Fastest route (minimize length)...")
fast_route = find_safe_route(G_risk, origin_point, dest_point, weight='length')

if safe_route is None or fast_route is None:
    print("\n⚠ Warning: Could not find route. Check origin/destination locations.")
else:
    print(f"\n✓ Routes calculated")
    print(f"  Safe route: {len(safe_route)} nodes")
    print(f"  Fast route: {len(fast_route)} nodes")

In [ ]:
# Convert routes to GeoDataFrames
safe_route_gdf = route_to_geodataframe(G_risk, safe_route)
fast_route_gdf = route_to_geodataframe(G_risk, fast_route)

print("Route details:")
if safe_route_gdf is not None:
    print(f"  Safe route length: {safe_route_gdf.geometry.length.sum():.0f}m")
if fast_route_gdf is not None:
    print(f"  Fast route length: {fast_route_gdf.geometry.length.sum():.0f}m")

## 5. Route Comparison

In [ ]:
# Calculate metrics
safe_metrics = calculate_route_metrics(G_risk, safe_route)
fast_metrics = calculate_route_metrics(G_risk, fast_route)

# Compare
comparison_df = compare_routes(G_risk, {
    'safe': safe_route,
    'fastest': fast_route
})

print("\nROUTE COMPARISON")
print("="*70)
print(comparison_df.to_string(index=False))

# Risk reduction
reduction = calculate_risk_reduction(safe_metrics, fast_metrics)

print("\nRISK REDUCTION ANALYSIS")
print("="*70)
print(f"Risk reduction: {reduction['risk_reduction_pct']:.1f}%")
print(f"Absolute risk reduction: {reduction['absolute_risk_reduction']:.4f}")
print(f"Length increase: {reduction['length_difference_m']:.0f}m ({reduction['length_increase_pct']:.1f}%)")
print(f"\nSafe route risk: {reduction['safe_route_risk']:.4f}")
print(f"Fast route risk: {reduction['fast_route_risk']:.4f}")

## 6. Interactive Map Visualization

In [ ]:
# Create base map
center_lat = (origin_point[0] + dest_point[0]) / 2
center_lon = (origin_point[1] + dest_point[1]) / 2

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles='OpenStreetMap'
)

print("Creating interactive map...")

In [ ]:
# Add risk heatmap layer
tiles_with_risk = tiles_gdf.merge(risk_df, on='tile_id', how='left').fillna(0)

folium.Choropleth(
    geo_data=tiles_with_risk,
    data=tiles_with_risk,
    columns=['tile_id', 'risk_prob'],
    key_on='feature.properties.tile_id',
    fill_color='YlOrRd',
    fill_opacity=0.5,
    line_opacity=0.1,
    legend_name='Crime Risk (Next 4 Hours)',
    name='Risk Heatmap'
).add_to(m)

print("✓ Risk heatmap added")

In [ ]:
# Add safe route (green)
if safe_route_gdf is not None:
    folium.GeoJson(
        safe_route_gdf,
        name='Safe Route',
        style_function=lambda x: {
            'color': 'green',
            'weight': 6,
            'opacity': 0.8
        },
        tooltip=f"Safe Route ({safe_metrics['length']:.0f}m)"
    ).add_to(m)
    print("✓ Safe route added (green)")

In [ ]:
# Add fast route (blue, dashed)
if fast_route_gdf is not None:
    folium.GeoJson(
        fast_route_gdf,
        name='Fastest Route',
        style_function=lambda x: {
            'color': 'blue',
            'weight': 6,
            'opacity': 0.8,
            'dashArray': '10, 5'
        },
        tooltip=f"Fastest Route ({fast_metrics['length']:.0f}m)"
    ).add_to(m)
    print("✓ Fastest route added (blue, dashed)")

In [ ]:
# Add origin/destination markers
folium.Marker(
    location=[origin_point[0], origin_point[1]],
    popup=f"<b>Origin</b><br>{origin_address if 'origin_address' in locals() else 'Start'}",
    icon=folium.Icon(color='green', icon='play')
).add_to(m)

folium.Marker(
    location=[dest_point[0], dest_point[1]],
    popup=f"<b>Destination</b><br>{dest_address if 'dest_address' in locals() else 'End'}",
    icon=folium.Icon(color='red', icon='stop')
).add_to(m)

print("✓ Markers added")

In [ ]:
# Add layer control
folium.LayerControl().add_to(m)

print("\n✓ Map ready!")
print("\nMap legend:")
print("  🟢 Green solid line = Safe route (minimizes risk)")
print("  🔵 Blue dashed line = Fastest route (minimizes distance)")
print("  🟥 Red heatmap = Crime risk intensity")

In [ ]:
# Display map
m

## 7. Export Routes

In [ ]:
# Export safe route as GeoJSON
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = f'../outputs/routes/safe_route_{timestamp}.geojson'

Path('../outputs/routes').mkdir(parents=True, exist_ok=True)

if safe_route_gdf is not None:
    export_route_geojson(safe_route_gdf, output_path)
    print(f"✓ Safe route exported to: {output_path}")
else:
    print("⚠ No route to export")

In [ ]:
# Save map as HTML
map_path = f'../outputs/routes/map_{timestamp}.html'
m.save(map_path)
print(f"✓ Interactive map saved to: {map_path}")

## Summary

**Guardian Route Analysis Complete**

This notebook:
- ✅ Loaded trained Cynet model
- ✅ Predicted 4-hour crime risk across Denver
- ✅ Computed safe route (risk-minimizing)
- ✅ Compared with fastest route
- ✅ Generated interactive visualization
- ✅ Exported routes as GeoJSON

### Key Findings

- **Safe route** reduces risk by avoiding high-crime areas
- Trade-off: Slightly longer distance for significantly lower risk
- Risk predictions update based on departure time

### Next Steps

- Try different origin/destination pairs
- Compare routes at different times of day
- Analyze hotspots identified by the model

---

**Note:** This is a research prototype for educational purposes. Route suggestions should not be considered safety guarantees.